# ADS / IDS Annotation Tool

**Everything happens here** — listen to segments and label them without leaving this notebook.

### Workflow
1. **Cell 2** — Configure participant & speaker  
2. **Cell 3** — Load audio & detect segments  
3. **Cell 4** — Annotate: plays each segment, you type `0` / `1` / `2` and press Enter  
4. **Cell 5** — Export labeled ADS & IDS WAV files

### Labels
| Input | Label |
|-------|-------|
| `0` | Other |
| `1` | ADS (Adult-Directed Speech) |
| `2` | IDS (Infant-Directed Speech) |
| `r` | Replay current segment |
| `b` | Go back one segment |
| `q` | Save & stop (resume later) |

In [ ]:
# ── Configuration ──────────────────────────────────────────────
PARTICIPANT_ID = "ABAN141223"       # ← change this
SPEAKER        = "female"           # "female" or "male"

# Paths (no need to change)
CLASSIFIED_ROOT  = "/scratch/users/arunps/hindibabynet/audio_classified"
ANNOTATION_ROOT  = "/scratch/users/arunps/hindibabynet/annotations"

# Segment detection tuning
SILENCE_THRESH_DB = -40   # frames below this RMS are silence
MIN_SILENCE_SEC   = 0.08  # min silence gap to split on
MIN_SEGMENT_SEC   = 0.25  # drop segments shorter than this
MAX_SEGMENT_SEC   = 15.0  # split segments longer than this

In [ ]:
import csv, wave, time
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import IPython.display as ipd
from IPython.display import display, clear_output

LABEL_MAP = {0: "Other", 1: "ADS", 2: "IDS"}

# ── Read WAV ──────────────────────────────────────────────────
def read_wav(path):
    with wave.open(str(path), "rb") as wf:
        sr = wf.getframerate()
        raw = wf.readframes(wf.getnframes())
    return np.frombuffer(raw, dtype=np.int16).astype(np.float32) / 32768.0, sr

# ── Segment detection ────────────────────────────────────────
def _rms_frames(audio, sr, frame_ms=20):
    frame_len = int(sr * frame_ms / 1000)
    n = len(audio) // frame_len
    if n == 0: return np.array([])
    rms = np.sqrt(np.mean(audio[:n*frame_len].reshape(n, frame_len)**2, axis=1) + 1e-12)
    return (20 * np.log10(rms + 1e-12)).astype(np.float32)

def detect_segments(audio, sr, thresh_db=-40, min_sil=0.08, min_seg=0.25, max_seg=15.0, frame_ms=20):
    db = _rms_frames(audio, sr, frame_ms)
    if len(db) == 0: return []
    frame_len = int(sr * frame_ms / 1000)
    is_sil = db < thresh_db
    min_sil_f = max(1, int(min_sil * 1000 / frame_ms))

    sil_regions = []
    in_sil, sil_s = False, 0
    for i, s in enumerate(is_sil):
        if s and not in_sil: in_sil, sil_s = True, i
        elif (not s) and in_sil:
            in_sil = False
            if i - sil_s >= min_sil_f: sil_regions.append((sil_s, i))
    if in_sil and len(is_sil) - sil_s >= min_sil_f:
        sil_regions.append((sil_s, len(is_sil)))

    total_dur = len(audio) / sr
    segs = []
    if not sil_regions:
        segs.append((0.0, total_dur))
    else:
        first = sil_regions[0][0] * frame_len / sr
        if first > 0: segs.append((0.0, first))
        for i in range(len(sil_regions)-1):
            s = sil_regions[i][1] * frame_len / sr
            e = sil_regions[i+1][0] * frame_len / sr
            if e > s: segs.append((s, e))
        last = sil_regions[-1][1] * frame_len / sr
        if last < total_dur: segs.append((last, total_dur))

    segs = [(s, e) for s, e in segs if e - s >= min_seg]

    # Split long segments
    final = []
    for s, e in segs:
        if e - s <= max_seg:
            final.append((s, e))
        else:
            final.extend(_split(audio, sr, s, e, max_seg, frame_ms))
    return final

def _split(audio, sr, start, end, max_s, frame_ms):
    if end - start <= max_s: return [(start, end)]
    seg = audio[int(start*sr):int(end*sr)]
    db = _rms_frames(seg, sr, frame_ms)
    if len(db) < 3: return [(start, end)]
    n = len(db); lo, hi = n//5, n - n//5
    sd = db[lo:hi]
    if len(sd) == 0: return [(start, end)]
    sp = lo + int(np.argmin(sd))
    split_sec = start + sp * int(sr * frame_ms / 1000) / sr
    return _split(audio, sr, start, split_sec, max_s, frame_ms) + \
           _split(audio, sr, split_sec, end, max_s, frame_ms)

# ── CSV persistence ──────────────────────────────────────────
def csv_path(pid, speaker):
    return Path(ANNOTATION_ROOT) / pid / f"{pid}_{speaker}_annotations.csv"

def load_ann(pid, speaker, segments):
    p = csv_path(pid, speaker)
    if not p.exists(): return {}
    ann = {}
    with open(p, newline="") as f:
        for row in csv.DictReader(f):
            ann[int(row["segment_index"])] = int(row["label"])
    return ann

def save_ann(pid, speaker, segments, annotations):
    p = csv_path(pid, speaker)
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(p, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["segment_index", "start_sec", "end_sec", "duration_sec", "label", "label_name"])
        for idx in sorted(annotations.keys()):
            if idx < len(segments):
                s, e = segments[idx]
                lab = annotations[idx]
                w.writerow([idx, f"{s:.4f}", f"{e:.4f}", f"{e-s:.4f}", lab, LABEL_MAP[lab]])
    return p

# ── Load audio & detect segments ─────────────────────────────
wav_path = Path(CLASSIFIED_ROOT) / PARTICIPANT_ID / f"{PARTICIPANT_ID}_main_{SPEAKER}.wav"
assert wav_path.exists(), f"File not found: {wav_path}"

print(f"Loading {wav_path.name} ...")
audio, sr = read_wav(wav_path)
dur = len(audio) / sr
print(f"Duration: {dur:.1f}s ({dur/60:.1f} min)  |  SR: {sr} Hz")

print(f"Detecting segments ...")
segments = detect_segments(audio, sr, SILENCE_THRESH_DB, MIN_SILENCE_SEC, MIN_SEGMENT_SEC, MAX_SEGMENT_SEC)
durs = [e - s for s, e in segments]
print(f"Found {len(segments)} segments  |  "
      f"min={min(durs):.2f}s  median={sorted(durs)[len(durs)//2]:.2f}s  max={max(durs):.2f}s")

# Load existing annotations
annotations = load_ann(PARTICIPANT_ID, SPEAKER, segments)
if annotations:
    print(f"Loaded {len(annotations)} existing annotations (will resume from where you left off)")
print(f"\n✓ Ready — run the next cell to start annotating.")

In [ ]:
# ── Annotate: listen + label in one loop ─────────────────────
# Re-run this cell to resume from where you left off.

total = len(segments)
# Find first unannotated segment
start_idx = 0
unannotated = [i for i in range(total) if i not in annotations]
if unannotated:
    start_idx = unannotated[0]
else:
    print(f"All {total} segments already annotated! Run the export cell below.")

n_ads = sum(1 for v in annotations.values() if v == 1)
n_ids = sum(1 for v in annotations.values() if v == 2)
n_oth = sum(1 for v in annotations.values() if v == 0)
print(f"Progress: {len(annotations)}/{total} annotated  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
print(f"Starting from segment {start_idx + 1}\n")
print("Labels:  0=Other  1=ADS  2=IDS  |  r=replay  b=back  q=save & stop\n")

idx = start_idx
try:
    while 0 <= idx < total:
        s, e = segments[idx]
        dur_s = e - s
        existing = annotations.get(idx)
        tag = f"  [current: {LABEL_MAP[existing]}]" if existing is not None else ""

        # Extract segment audio
        s_samp = max(0, int(s * sr))
        e_samp = min(len(audio), int(e * sr))
        chunk = audio[s_samp:e_samp]

        clear_output(wait=True)
        n_ads = sum(1 for v in annotations.values() if v == 1)
        n_ids = sum(1 for v in annotations.values() if v == 2)
        n_oth = sum(1 for v in annotations.values() if v == 0)
        print(f"[{idx+1}/{total}]  {s:.2f}s – {e:.2f}s  ({dur_s:.2f}s){tag}")
        print(f"Progress: {len(annotations)}/{total}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
        print("─" * 50)

        # Play audio (autoplay in browser)
        display(ipd.Audio(chunk, rate=sr, autoplay=True))

        while True:
            raw = input("Label (0/1/2/r/b/q): ").strip().lower()
            if raw == "q":
                save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations)
                clear_output(wait=True)
                n_ads = sum(1 for v in annotations.values() if v == 1)
                n_ids = sum(1 for v in annotations.values() if v == 2)
                n_oth = sum(1 for v in annotations.values() if v == 0)
                print(f"✓ Saved {len(annotations)}/{total} annotations.  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
                print(f"  CSV: {csv_path(PARTICIPANT_ID, SPEAKER)}")
                print(f"  Re-run this cell to resume.")
                raise StopIteration
            elif raw == "r":
                clear_output(wait=True)
                print(f"[{idx+1}/{total}]  {s:.2f}s – {e:.2f}s  ({dur_s:.2f}s)  (replaying...)")
                print(f"Progress: {len(annotations)}/{total}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
                print("─" * 50)
                display(ipd.Audio(chunk, rate=sr, autoplay=True))
                continue
            elif raw == "b":
                if idx > 0:
                    idx -= 1
                break
            elif raw in ("0", "1", "2"):
                annotations[idx] = int(raw)
                idx += 1
                # Auto-save every 10
                if len(annotations) % 10 == 0:
                    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations)
                break
            else:
                print("  Invalid. Use 0, 1, 2, r, b, or q.")

    # Finished all segments
    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations)
    clear_output(wait=True)
    n_ads = sum(1 for v in annotations.values() if v == 1)
    n_ids = sum(1 for v in annotations.values() if v == 2)
    n_oth = sum(1 for v in annotations.values() if v == 0)
    print(f"🎉 All {total} segments annotated!  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
    print(f"CSV: {csv_path(PARTICIPANT_ID, SPEAKER)}")
    print(f"Run the export cell below to create separate WAV files.")

except StopIteration:
    pass
except KeyboardInterrupt:
    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations)
    print(f"\nInterrupted — saved {len(annotations)} annotations. Re-run to resume.")

In [ ]:
# ── Export: create separate ADS / IDS / Other WAV files ──────
# Run this after annotation is complete (or partially complete).

def write_wav16(path, data, sr):
    path.parent.mkdir(parents=True, exist_ok=True)
    pcm = np.clip(data * 32768.0, -32768, 32767).astype(np.int16)
    with wave.open(str(path), "wb") as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr)
        wf.writeframes(pcm.tobytes())

out_dir = Path(ANNOTATION_ROOT) / PARTICIPANT_ID
gap = np.zeros(int(sr * 0.15), dtype=np.float32)

n_ann = len(annotations)
n_ads = sum(1 for v in annotations.values() if v == 1)
n_ids = sum(1 for v in annotations.values() if v == 2)
n_oth = sum(1 for v in annotations.values() if v == 0)
print(f"Annotations: {n_ann}/{len(segments)}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}\n")

for label_code, label_name in LABEL_MAP.items():
    pieces = []
    for idx, lab in sorted(annotations.items()):
        if lab != label_code or idx >= len(segments): continue
        s, e = segments[idx]
        chunk = audio[max(0, int(s*sr)):min(len(audio), int(e*sr))]
        if len(chunk) > 0:
            pieces.append(chunk)
            pieces.append(gap)
    if pieces:
        combined = np.concatenate(pieces)
        out_path = out_dir / f"{PARTICIPANT_ID}_{SPEAKER}_{label_name}.wav"
        write_wav16(out_path, combined, sr)
        out_dur = len(combined) / sr
        print(f"  {label_name:6s} → {out_path}  ({out_dur:.1f}s)")
    else:
        print(f"  {label_name:6s} → (no segments)")

print(f"\n✓ Export complete.")